# Parte 5. Análisis de datos interactivo

## Introducción


Este notebook introduce el análisis de datos interactivo de datos en BigQuery
utilizando un notebook de Jupyter gestionado por Vertex AI Workbench, el
cual es un entorno basado en notebooks de Jupyter que se proporciona a
través de instancias de máquina virtual (VM) con funciones que admiten
todo el flujo de trabajo de la ciencia de datos.

Puedes usar el entorno basado en notebooks de Vertex AI Workbench para
consultar y explorar datos, desarrollar y entrenar un modelo, y ejecutar
tu código como parte de un pipeline (canalización).

Esta celda, por ejemplo, es una celda de markdown. Es por eso que estás
viendo texto. La celda que sigue es una celda de código Python. La salida
de esa celda es lo que se imprime desde ella.

In [ ]:
a = 3
b = a + 5
print(f"a={a} b={b}")

## Ruta relativa


Este notebook se creó en la carpeta 05_bqnotebook del repositorio de git.
Por lo tanto, es posible que veas una ruta que termine en eso. Pero la ruta
comenzará con /home/jupyter, que está mapeada a una carpeta local si se
ejecuta en un contenedor, de otro modo si estas en tu dispositivo local
veras la ruta local al archivo

In [ ]:
!pwd

## ¿Qué está instalado?


¿Usamos !pip o %pip?

La primera opción ejecuta el comando en la shell y si no tenemos activado el ambiente ahi podría darnos resultados errados.

La segunda opción ejecuta pip por una función mágica, la cual ejecuta el
gestor de paquetes dentro del kernel(núcleo) actual 

In [ ]:
%pip freeze

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from google.cloud import bigquery

## Instalando dependencias

Las dependencias de Python regulares se pueden instalar utilizando pip

In [ ]:
%pip install pytz

## Juypter magic

In [ ]:
%%html
Esta celda imprimirá un `string` de <b> HTML </b>.

El cell magic `%%bigquery` nos retorna los resultados de la consulta en
SQL en un DataFrame de pandas.


El sintaxis de la celda mágica es el siguiente:

```
%%bigquery [<destination_var>] [--project <project>] [--use_legacy_sql]
           [--verbose] [--params <params>] <query>
```

Obs. si nos encontramos en un ambiente local, antes de ejecutar la celda
debemos ejecutar la linea mágica load_ext la cual carga las funciones
mágicas por su nombre de módulo.

In [5]:
%load_ext google.cloud.bigquery

In [ ]:
%%bigquery
SELECT
    COUNTIF(arr_delay >= 15)/COUNT(arr_delay) AS frac_delayed
FROM
    dsongcp.flights_tzcorr

## Acceso a llamadas a BigQuery

También podemos realizar llamadas a BigQuery directamente con la biblioteca
de Python:

In [ ]:
bq = bigquery.Client()

In [ ]:
sql = """
SELECT
  COUNTIF(arr_delay >= 15)/COUNT(arr_delay) AS frac_delayed
FROM dsongcp.flights_tzcorr
"""
bq.query(sql).to_dataframe()

Grafiquemos una Función de Distribución de Probabilidad (PDF o Probability
Distribution Function) de diferentes retrasos de llegada. En un Notebook,
podemos asignar la salida de una consulta de la celda mágica a una variable,
en este caso df, como vimos en la sintáxis de la celda mágica %%biggquery:

In [ ]:
%%bigquery df
SELECT
    ARR_DELAY, DEP_DELAY
FROM
    dsongcp.flights_tzcorr
WHERE
    DEP_DELAY >= 10

In [ ]:
type(df)

In [ ]:
df.describe()

In [ ]:
# predefined style {darkgrid=Default, whitegrid, dark, white, ticks}
sns.set_style(style="whitegrid")
# sns.set_theme(font_scale=1.5)  # Default font_scale=1
ax = sns.violinplot(data=df, x="ARR_DELAY", inner="box", orient="h")
ax.axes.set_xlim(-50, 300)

## Visualizando distribuciones

In [ ]:
%%bigquery df
SELECT ARR_DELAY, DEP_DELAY
FROM dsongcp.flights_tzcorr

In [ ]:
df.describe()

In [ ]:
df['ontime'] = df['DEP_DELAY'] < 10
df[df['ARR_DELAY'] > 0].head()

In [ ]:
sns.set_style("whitegrid")
ax = sns.violinplot(data=df, x='ARR_DELAY', y='ontime', inner='box', orient='h')
ax.set_xlim(-50, 200);

Podemos modificar el argumento gridsize, el cual es el número de puntos
en la malla discreta usada para evaluar la estimación de densidad del kernel
para obtener un gráfico mas "suave" a cambio de un mayor tiempo.

In [ ]:
ax = sns.violinplot(data=df, x='ARR_DELAY', y='ontime', 
                    inner='box', orient='h', gridsize=1000)
ax.set_xlim(-50, 200)

## Valores atípicos

La siguiente consulta nos entrega el retraso de llegada promedio y el numero
de vuelos, asociados con cada valor de retraso de llegada

In [ ]:
%%bigquery depdelay
SELECT
    DEP_DELAY,
    AVG(ARR_DELAY) AS arrival_delay,
    COUNT(ARR_DELAY) AS numflights
FROM
    dsongcp.flights_tzcorr
GROUP BY
    DEP_DELAY
ORDER BY
    DEP_DELAY

El número de registros de cada grupo lo vemos a continuación

In [ ]:
len(depdelay)

Obtenemos mas de 1800 valores únicos, esto quiere decir que algo pasa con los
datos y tenemos valores fuera de lo común.

In [ ]:
depdelay[:5]

In [ ]:
depdelay[55:60]

Las primeras filas tienen pocos vuelos, sin embargo los valores de retraso
pequeños tienen cantidades considerables, los valores poco comunes que
son una proporción pequeña con respecto al total de los datos pueden ser
probablemente ignorados sin afectar a nuestro modelo.

## Descartando valores atípicos


Como remover dichos valores atípicos? Hay dos grandes maneras para filtrar
los datos, una sería basado en la variable dep_delay (retraso de salida)
manteniendo solo los valores que cumplen una condición, por ejemplo
dep_delay > -15. Un segundo método sería filtrar los datos basado en el
numero de vuelos.

El segundo método usar un filtro de control de calidad que está basado
en remover datos para los cuales tenemos ejemplos insuficientes es preferible.

Obs. Este es un punto importante que destaca la diferencia clave entre la
estadística en conjuntos de datos "normales" y la estadística en grandes
conjuntos de datos. El enfoque fundamental para abordar problemas cambia cuando
los conjuntos de datos crecen lo suficiente. La forma en que detectamos valores
atípicos es solo un ejemplo de esto.

Por ejemplo para conjuntos de datos que tienen cientos o miles de ejemplos,
filtraríamos valores fuera de $\mu \pm 3 \sigma$ (donde $\mu$ es la media y
$\sigma$ es la desviación estándar)

In [ ]:
%%capture
%%bigquery limits
SELECT
    AVG(DEP_DELAY) - 3*STDDEV(DEP_DELAY) AS filter_min,
    AVG(DEP_DELAY) + 3*STDDEV(DEP_DELAY) AS filter_max
FROM
    dsongcp.flights_tzcorr;

In [ ]:
limits

Esto nos lleva al rango de $[-157, 185]$ minutos por ende podriamos filtrar con una sentencia `WHERE dep_delay BETWEEN -157 AND 185`.

Obs. Este filtro está basado en el supuesto de que la distribución de los
retrasos de vuelos es "Normal" (Gaussiana).

Otro ejemplo que evita este supuesto, sería usar percentiles omitiendo algún
porcentaje de los datos digamos 5% de los valores mas altos y bajos.

In [10]:
%%capture
%%bigquery quantiles
SELECT
APPROX_QUANTILES(DEP_DELAY, 20)
FROM
dsongcp.flights_tzcorr;

El DataFrame devuelto por bigquery es de una fila y una columna, su contenido
es un array de 21 floats que contienen los 20 cuantiles aproximados, calculados
por la función `APPROX_QUANTILES` para `DEP_DELAY` por ende consideramos el
segundo y penúltimo elemento del array.

In [16]:
quantiles.iat[0,0]

array([-7.200e+01, -1.000e+01, -8.000e+00, -7.000e+00, -6.000e+00,
       -5.000e+00, -5.000e+00, -4.000e+00, -3.000e+00, -2.000e+00,
       -2.000e+00, -1.000e+00,  0.000e+00,  3.000e+00,  6.000e+00,
        1.100e+01,  1.700e+01,  2.700e+01,  4.400e+01,  8.300e+01,
        4.413e+03])

Por lo tanto el intervalo entregado por este filtro es $[-10,83]$

En conclusión en conjuntos de datos que contienen cientos de miles a millones
de ejemplos, establecer umbrales en los datos de entrada según su valor es
peligroso porque puedes estar descartando matices valiosos. Si hay suficientes
ejemplos de un retraso de 150 minutos, vale la pena modelar dicho valor
independientemente de lo lejos que esté de la media. A medida que nuestro
conjunto de datos vaya creciendo los valores particulares se van haciendo menos
frecuentes.

## Filtrado de datos por frecuencia de ocurrencia

Para filtrar el conjunto de datos basado en la frecuencia de ocurrencia,
primero necesitamos calcular la frecuencia de ocurrencia y luego aplicar
un umbral a los datos en función de esta.

In [24]:
%%capture
%%bigquery occ
SELECT
    DEP_DELAY,
    AVG(ARR_DELAY) AS arrival_delay,
    STDDEV(ARR_DELAY) AS stddev_arr_delay,
    COUNT(ARR_DELAY) AS numflights
FROM
    dsongcp.flights_tzcorr
GROUP BY
    DEP_DELAY
HAVING
    numflights > 370
ORDER BY
    DEP_DELAY


In [ ]:
occ.head()


¿Por qué establecer un umbral de 370 vuelos? Este número se obtiene de una guía
llamada regla de los tres sigmas, que tradicionalmente es el rango dentro del
cual consideramos que "casi todos los valores" se encuentran. Si asumimos (por
ahora; lo verificaremos pronto) que, para cualquier retraso de salida, los
retrasos de llegada están distribuidos normalmente, podemos hablar de hechos que
son verdaderos para "casi todos los vuelos" si nuestro tamaño de población es
lo suficientemente grande. Debido a que el 99.73% de los valores en una
distribución gaussiana se encuentran dentro de los límites de tres sigmas,
filtrar nuestro conjunto de datos para que tengamos al menos $1/(1 - 0,9973)=370$
ejemplos de cada valor de entrada es una regla general que logra esto.

### ¿Que tan distintos son los resultados si consideramos un umbral distinto?

Podemos ver el numero de vuelos que son removidos según diferentes umbrales
de control de calidad, observando la pendiente del modelo lineal entre el
retraso de llegada y el retraso de salida.

Obs. Estrictamente no es un modelo lineal si no que una simplificacion de este
ya que hay supuestos con respecto al error e intercepto.

In [25]:
%%capture
%%bigquery df
DECLARE total_flights INT64;
SET total_flights = (
    SELECT COUNT(*) FROM dsongcp.flights_tzcorr
);

CREATE TEMPORARY FUNCTION linear_fit(NUM_TOTAL INT64, THRESH INT64)
RETURNS STRUCT<thresh INT64, num_removed INT64, lm FLOAT64>
AS ((
    SELECT AS STRUCT
        THRESH,
        (NUM_TOTAL - SUM(numflights)) AS num_removed,
        ROUND(AVG(arrival_delay * numflights) / AVG(dep_delay * numflights), 2) AS lm
    FROM
    (
        SELECT
            DEP_DELAY,
            AVG(ARR_DELAY) AS arrival_delay,
            STDDEV(ARR_DELAY) AS stddev_arrival_delay,
            COUNT(ARR_DELAY) AS numflights
        FROM
            dsongcp.flights_tzcorr
        GROUP BY
            DEP_DELAY
    )
    WHERE numflights > THRESH
))
;

SELECT linear_fit(total_flights, 1000) stats
UNION ALL SELECT linear_fit(total_flights, 500)
UNION ALL SELECT linear_fit(total_flights, 370)
UNION ALL SELECT linear_fit(total_flights, 300)
UNION ALL SELECT linear_fit(total_flights, 200)
UNION ALL SELECT linear_fit(total_flights, 100)
UNION ALL SELECT linear_fit(total_flights, 22)
UNION ALL SELECT linear_fit(total_flights, 10)
UNION ALL SELECT linear_fit(total_flights, 5)
ORDER BY stats.thresh DESC

In [26]:
df['stats'].map(lambda x: (x['thresh'], x['num_removed'], x['lm']))

0    (1000, 234025, 0.4)
1    (500, 194535, 0.47)
2    (370, 185980, 0.49)
3     (300, 179887, 0.5)
4    (200, 169894, 0.52)
5    (100, 160616, 0.54)
6     (22, 148140, 0.57)
7      (10, 141085, 0.6)
8      (5, 140076, 0.61)
Name: stats, dtype: object

Podemos ver que la pendiente disminuye cada vez menos a medida que disminuimos
el umbral. Por lo tanto las diferencias en el modelo creado los umbrales de 300
, 370 o 500 son bastante menores. sin embargo el modelo es bastante diferente
si los umbrales fuesen 5 o 10 vuelos. El orden de magnitud del umbral importa
quizás no tanto su valor exacto.

## Arrival delay conditioned on departure delay

In [ ]:
%%bigquery depdelay
SELECT
    DEP_DELAY,
    AVG(ARR_DELAY) AS arrival_delay,
    STDDEV(ARR_DELAY) AS stddev_arrival_delay,
    COUNT(ARR_DELAY) AS numflights
FROM
    dsongcp.flights_tzcorr
GROUP BY
    DEP_DELAY
HAVING numflights > 370
ORDER BY DEP_DELAY

In [ ]:
depdelay[:5]

In [ ]:
ax = depdelay.plot(kind='line', x='DEP_DELAY', 
              y='arrival_delay', yerr='stddev_arrival_delay')

In [ ]:
Z_30 = 0.52
depdelay['arr_delay_30'] = (Z_30 * depdelay['stddev_arrival_delay']) \
             + depdelay['arrival_delay']

ax = plt.axes()
depdelay.plot(kind='line', x='DEP_DELAY', y='arr_delay_30',
              ax=ax, ylim=(0,30), xlim=(0,30), legend=False)
ax.set_xlabel('Departure Delay (minutes)')
ax.set_ylabel('> 30% prob of this\n Arrival Delay (minutes)');

x = np.arange(0, 30)
y = np.ones_like(x) * 15
ax.plot(x, y, 'r.');

y = np.arange(0, 30)
x = np.ones_like(y) * 13
ax.plot(x, y, 'g.');

In [ ]:
%%bigquery depdelay
SELECT
    DEP_DELAY,
    APPROX_QUANTILES(ARR_DELAY, 101)[OFFSET(70)] AS arrival_delay,
    COUNT(ARR_DELAY) AS numflights
FROM
    dsongcp.flights_tzcorr
GROUP BY
    DEP_DELAY
HAVING numflights > 370
ORDER BY DEP_DELAY

In [ ]:
ax = plt.axes()
depdelay.plot(kind='line', x='DEP_DELAY', y='arrival_delay',
              ax=ax, ylim=(0,30), xlim=(0,30), legend=False)
ax.set_xlabel('Departure Delay (minutes)')
ax.set_ylabel('> 30% prob of this\n Arrival Delay (minutes)');

x = np.arange(0, 30)
y = np.ones_like(x) * 15
ax.plot(x, y, 'r.');

y = np.arange(0, 30)
x = np.ones_like(y) * 16
ax.plot(x, y, 'g.');

## Creating training/evaluation dataset

In [ ]:
%%bigquery
SELECT
  FL_DATE,
  IF(ABS(MOD(FARM_FINGERPRINT(CAST(FL_DATE AS STRING)), 100)) < 70,
     'True', 'False') AS is_train_day
FROM (
  SELECT
    DISTINCT(FL_DATE) AS FL_DATE
  FROM
    dsongcp.flights_tzcorr)
ORDER BY
  FL_DATE
LIMIT 5

In [ ]:
%%bigquery
CREATE OR REPLACE TABLE dsongcp.trainday AS

SELECT
  FL_DATE,
  IF(ABS(MOD(FARM_FINGERPRINT(CAST(FL_DATE AS STRING)), 100)) < 70,
     'True', 'False') AS is_train_day
FROM (
  SELECT
    DISTINCT(FL_DATE) AS FL_DATE
  FROM
    dsongcp.flights_tzcorr)
ORDER BY
  FL_DATE

In [ ]:
%%bigquery depdelay
SELECT
    DEP_DELAY,
    APPROX_QUANTILES(ARR_DELAY, 101)[OFFSET(70)] AS arrival_delay,
    COUNT(ARR_DELAY) AS numflights
FROM
    dsongcp.flights_tzcorr
JOIN dsongcp.trainday USING(FL_DATE)
WHERE is_train_day = 'True'
GROUP BY
    DEP_DELAY
HAVING numflights > 370
ORDER BY DEP_DELAY

In [ ]:
ax = plt.axes()
depdelay.plot(kind='line', x='DEP_DELAY', y='arrival_delay',
              ax=ax, ylim=(0,30), xlim=(0,30), legend=False)
ax.set_xlabel('Departure Delay (minutes)')
ax.set_ylabel('> 30% prob of this\n Arrival Delay (minutes)');

x = np.arange(0, 30)
y = np.ones_like(x) * 15
ax.plot(x, y, 'r.');

y = np.arange(0, 30)
x = np.ones_like(y) * 16
ax.plot(x, y, 'g.');

In [ ]:
%%bigquery df_eval
SELECT
  SUM(IF(DEP_DELAY < 16
      AND arr_delay < 15, 1, 0)) AS correct_nocancel,
  SUM(IF(DEP_DELAY < 16
      AND arr_delay >= 15, 1, 0)) AS wrong_nocancel,
  SUM(IF(DEP_DELAY >= 16
      AND arr_delay < 15, 1, 0)) AS wrong_cancel,
  SUM(IF(DEP_DELAY >= 16
      AND arr_delay >= 15, 1, 0)) AS correct_cancel
FROM (
  SELECT
    DEP_DELAY,
    ARR_DELAY
  FROM
    dsongcp.flights_tzcorr
  JOIN dsongcp.trainday USING(FL_DATE)
  WHERE is_train_day = 'False' 
)

In [ ]:
print(df_eval['correct_nocancel'] /
      (df_eval['correct_nocancel'] + df_eval['wrong_nocancel']))
print(df_eval['correct_cancel'] / 
      (df_eval['correct_cancel'] + df_eval['wrong_cancel']))

In [ ]:
df_eval.head()

In [ ]:
%%bigquery df_eval
SELECT
  SUM(IF(DEP_DELAY = 15
      AND arr_delay < 15, 1, 0)) AS correct_nocancel,
  SUM(IF(DEP_DELAY = 15
      AND arr_delay >= 15, 1, 0)) AS wrong_nocancel,
  SUM(IF(DEP_DELAY = 16
      AND arr_delay < 15, 1, 0)) AS wrong_cancel,
  SUM(IF(DEP_DELAY = 16
      AND arr_delay >= 15, 1, 0)) AS correct_cancel
FROM (
  SELECT
    DEP_DELAY,
    ARR_DELAY
  FROM
    dsongcp.flights_tzcorr
  JOIN dsongcp.trainday USING(FL_DATE)
  WHERE is_train_day = 'False' 
)

In [ ]:
df_eval.head()

In [ ]:
print(df_eval['correct_nocancel'] / (df_eval['correct_nocancel'] + df_eval['wrong_nocancel']))
print(df_eval['correct_cancel'] / (df_eval['correct_cancel'] + df_eval['wrong_cancel']))

Copyright 2021 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.